In [ ]:
import tkinter as tk
from PIL import Image, ImageTk, ImageDraw
from openai import OpenAI
import requests
from io import BytesIO
import os

######################################
##     ENTER API KEY BELOW     ##
######################################

# Initialize the OpenAI client
client = OpenAI(api_key="ENTER API KEY")

# Scale factor (50% scaling)
SCALE_FACTOR = 0.5

# Flag to track whether we are in edit mode
edit_mode_enabled = False

# Paths for the images
latest_image_path = "masked_images/original_image.png"  # The latest edited image or original image
unaltered_image_path = "masked_images/unaltered_image.png"  # Unaltered image without transparency

# Create directory for saving images
if not os.path.exists("masked_images"):
    os.makedirs("masked_images")

# Load the image from a URL and display it, scaling down by 50%
def generate_image(prompt, model="dall-e-2", size="1024x1024", n=1, quality="standard"):
    try:
        # Call the OpenAI API to generate an image
        response = client.images.generate(
            model=model,
            prompt=prompt,
            size=size,
            quality=quality,
            n=n
        )

        # Get the image URL from the response
        image_url = response.data[0].url
        return image_url
    except Exception as e:
        print(f"Failed to generate image: {e}")
        return None

# Display the image and show the mask creation button
def display_image(img):
    global SCALE_FACTOR  # Ensure the scale factor is accessible in this function
    try:
        # Scale the image down by 50% (i.e., to 50% of its original size)
        original_width, original_height = img.size
        new_width = int(original_width * SCALE_FACTOR)
        new_height = int(original_height * SCALE_FACTOR)
        img = img.resize((new_width, new_height), Image.ANTIALIAS)

        # Adjust the canvas size to match the new image size
        canvas.config(width=new_width, height=new_height)

        # Display the image on the canvas
        img_tk = ImageTk.PhotoImage(img)
        canvas.img_tk = img_tk
        canvas.create_image(0, 0, anchor=tk.NW, image=img_tk)

        # Resize the window to fit the image plus extra space for the buttons
        root.geometry(f"{new_width}x{new_height + 250}")  # Adjust window size for better button visibility

        # Show the button for highlighting and transparency
        highlight_button.pack()

    except Exception as e:
        print(f"Failed to display image: {e}")

# Function to reset the image to its original state or the latest unaltered edited image
def reset_image():
    global img, edit_mode_enabled
    img = Image.open(unaltered_image_path).convert("RGBA")  # Open the last unaltered image (without transparency)
    display_image(img)
    
    # Reset the text box for prompt input
    # prompt_box.delete("1.0", tk.END)  # Clear the text box
    
    # Reset the mode to generate a new image instead of editing
    edit_mode_enabled = False
    
    # Hide the "New Image" button
    new_image_button.pack_forget()

# Function to reset to new image creation mode
def reset_to_new_image_mode():
    global edit_mode_enabled, latest_image_path, unaltered_image_path
    edit_mode_enabled = False
    latest_image_path = "masked_images/original_image.png"  # Reset to original image
    unaltered_image_path = latest_image_path  # Unaltered should also reset to the original image
    print("New image creation mode enabled. Subsequent prompts will generate a new image.")
    new_image_button.pack_forget()

# Function to create transparency in the circled area and enable editing
def make_transparent(start, end):
    global img, edit_mode_enabled, latest_image_path, unaltered_image_path

    # Save the current image as the unaltered image (before applying transparency)
    img.save(unaltered_image_path, "PNG")

    # Adjust the start and end points according to the scale factor
    start = (int(start[0] / SCALE_FACTOR), int(start[1] / SCALE_FACTOR))
    end = (int(end[0] / SCALE_FACTOR), int(end[1] / SCALE_FACTOR))

    # Calculate the midpoint between the start and end for the center of the circle
    center_x = (start[0] + end[0]) // 2
    center_y = (start[1] + end[1]) // 2
    radius = int(((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2) ** 0.5 / 2)

    # Open a copy of the latest image (this will be the mask after we apply transparency)
    mask_img = Image.open(latest_image_path).convert("RGBA")
    img_copy_data = mask_img.load()

    # Create a mask to apply transparency
    mask = Image.new("L", mask_img.size, 0)
    draw = ImageDraw.Draw(mask)
    draw.ellipse([center_x - radius, center_y - radius, center_x + radius, center_y + radius], fill=255)

    # Apply the transparency to the displayed image (img) and the mask image
    img_data = img.load()
    for y in range(mask_img.height):
        for x in range(mask_img.width):
            if mask.getpixel((x, y)) > 0:
                img_copy_data[x, y] = (*img_copy_data[x, y][:3], 0)  # Make the pixel transparent in the mask
                img_data[x, y] = (*img_data[x, y][:3], 0)  # Apply the transparency to the displayed image

    # Save the mask image (displayed image with transparency)
    mask_img.save("masked_images/mask.png", "PNG")

    # Save the latest edited image for further editing
    img.save("masked_images/edited_image.png", "PNG")

    # Update the latest image path to be the new edited image
    latest_image_path = "masked_images/edited_image.png"

    # Display the updated image with the transparent area
    display_image(img)

    # Enable edit mode for subsequent prompts
    edit_mode_enabled = True

    # Show the "New Image" button to switch back to new image generation
    new_image_button.pack()

# Function to enable highlight and transparency mode
def enable_highlight_mode():
    canvas.bind("<ButtonPress-1>", on_mouse_down)
    canvas.bind("<B1-Motion>", on_mouse_move)
    canvas.bind("<ButtonRelease-1>", on_mouse_up)
    reset_button.pack()

# Mouse events for drawing the circle on the image
start_x, start_y, circle_id = None, None, None

def on_mouse_down(event):
    global start_x, start_y, circle_id
    start_x, start_y = event.x, event.y
    circle_id = canvas.create_oval(start_x, start_y, start_x, start_y, outline="red")

def on_mouse_move(event):
    global circle_id
    if circle_id:
        canvas.coords(circle_id, start_x, start_y, event.x, event.y)

def on_mouse_up(event):
    global start_x, start_y, circle_id
    if circle_id:
        make_transparent((start_x, start_y), (event.x, event.y))
        canvas.delete(circle_id)
        circle_id = None

# Function to handle the image generation or editing from prompt
def on_send():
    global img, img_copy, latest_image_path, unaltered_image_path  # Ensure global variables are declared
    prompt = prompt_box.get("1.0", tk.END).strip()
    if prompt:
        if edit_mode_enabled:
            # Edit the existing image using the mask
            print("Editing image with the prompt:", prompt)
            try:
                response = client.images.edit(
                    model="dall-e-2",
                    image=open(latest_image_path, "rb"),  # Use the original or latest edited image
                    mask=open("masked_images/mask.png", "rb"),  # Use the displayed image with transparency as mask
                    prompt=prompt,
                    n=1,
                    size="1024x1024"
                )
                image_url = response.data[0].url

                # Load and display the edited image
                response = requests.get(image_url)
                img_data = response.content
                img = Image.open(BytesIO(img_data)).convert("RGBA")

                # Save the newly edited image as the latest image
                img.save("masked_images/edited_image.png", "PNG")
                latest_image_path = "masked_images/edited_image.png"
                unaltered_image_path = latest_image_path  # Keep this as the unaltered version

                # Display the newly edited image
                display_image(img)
            except Exception as e:
                print(f"Failed to edit image: {e}")
        else:
            # Generate a new image
            print("Generating new image with the prompt:", prompt)
            image_url = generate_image(prompt, model="dall-e-2", size="1024x1024", n=1, quality="standard")
            if image_url:
                response = requests.get(image_url)
                img_data = response.content
                img = Image.open(BytesIO(img_data)).convert("RGBA")
                img_copy = img.copy()  # Keep an original copy

                # Save the original image as the latest image for future edits
                img.save("masked_images/original_image.png", "PNG")
                latest_image_path = "masked_images/original_image.png"
                unaltered_image_path = latest_image_path  # Set this as the unaltered version for reset

                # Display the image
                display_image(img)

                # Hide the "New Image" button (not needed until after an edit)
                new_image_button.pack_forget()
    else:
        print("Input Required: Please enter a prompt.")

# Initialize the Tkinter window
root = tk.Tk()
root.title("Image Generator")
root.geometry("512x150")

# Create a textbox for the prompt input
prompt_box = tk.Text(root, height=4, width=50)
prompt_box.pack(pady=10)

# Create a send button for generating the image or submitting an edit
send_button = tk.Button(root, text="Send", command=on_send)
send_button.pack(pady=10)

# Create a canvas to display the image, canvas size will be adjusted dynamically
canvas = tk.Canvas(root)
canvas.pack()

# Button to initiate mouse highlight and mask creation
highlight_button = tk.Button(root, text="Highlight and Make Transparent", command=enable_highlight_mode)
highlight_button.pack_forget()  # Initially hidden

# Button to reset the image (will only show after highlighting starts)
reset_button = tk.Button(root, text="Reset Image", command=reset_image)
reset_button.pack_forget()

# Button to reset to new image creation mode (only visible after editing/masking)
new_image_button = tk.Button(root, text="New Image", command=reset_to_new_image_mode)
new_image_button.pack_forget()

# Start the Tkinter event loop
root.mainloop()